# Description

I want to compile an arbitrary quantum two-qubit gate to a valid 19Q program. The [allowed instructions](http://pyquil.readthedocs.io/en/latest/qpu_usage.html) are
* Single-qubit rotations around Z axis
* Single-qubit rotations around X axis by $\pm \pi / 2$
* Control-Z between nearest neighbours

In [[PRA 63, 062309]](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.63.062309), Appendix A, it is shown a method to decompose an arbitrary two-qubit unitary in the form
$$
U = \left(U_A \otimes U_B\right) N\left(\alpha,\beta,\gamma\right)  \left(U_C \otimes U_D\right)
$$
where $U_{A,B,C,D}$ are arbitrary single-qubit unitaries and 
$$
N\left(\alpha,\beta,\gamma\right) = e^{-i\left( \alpha \sigma_Z \otimes \sigma_Z + \beta \sigma_X \otimes \sigma_X +  \gamma \sigma_Y \otimes \sigma_Y \right)}
$$
is a two-qubit operation.

In [[PRA 69, 032315]](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.032315), it is shown a method to implement $N\left(\alpha,\beta,\gamma\right)$ using at most three CZs. 

In [1]:
import pylab as py
import qutip as qp
from pyquil.quil import Program
import pyquil.api as api
import pyquil.gates as gt
qvm = api.QVMConnection()
import math,cmath,itertools,scipy.linalg

## Finding $\left(U_A,U_B,U_C,U_D,\alpha,\beta,\gamma\right)$

A more involved problem seems to be the one to find the decomposition parameters such that
$$
U = \left(U_A \otimes U_B\right) e^{-i\left( \alpha \sigma_Z \otimes \sigma_Z + \beta \sigma_X \otimes \sigma_X +  \gamma \sigma_Y \otimes \sigma_Y \right)} \left(U_C \otimes U_D\right)
$$


In [2]:
def decomposition(unitary, index=[0,1]):
    """
    Evaluate the decomposition of an unitary
    described in Appendix A of PRA 63, 062309
    """
    if isinstance(unitary,list):
        unitary = py.array(unitary)
    if isinstance(unitary,py.ndarray):
        assert unitary.shape == (4,4) , u"'unitary' must be a 4x4 matrix"
        unitary = qp.Qobj(unitary,dims=[[2,2],[2,2]])
    assert isinstance(unitary,qp.qobj.Qobj), u"'unitary' must be convertible to a 4x4 quantum operator"
    assert unitary.dims == [[2,2],[2,2]], u"Dimension must be [[2,2],[2,2]]"
    
    Ua,Ub,Va,Vb,Udiag = decompositionCirac(unitary)
    
    for corrections,gates in decompositionTwoQubits(Udiag,index):

        generators = []
        generators.append(decompositionOneQubit(corections[0] * Ua,index[0]))
        generators.append(decompositionOneQubit(corections[1] * Ub,index[1]))
        generators.append(gates)
        generators.append(decompositionOneQubit(Va * corections[2],index[0]))
        generators.append(decompositionOneQubit(Vb * corections[3],index[1]))
        for prog in itertools.product(*generators):
            programs = [x[0] for x in prog]
            program = prog[0][0]
            for next_program in prog[1:]:
                program = program + next_program[0]
            yield program

    


In [229]:
magical_vectors = []
magical_vectors.append([1,0,0,1])
magical_vectors.append([-1j,0,0,1j])
magical_vectors.append([0,1,-1,0])
magical_vectors.append([0,-1j,-1j,0])
magical_basis = qp.Qobj(py.array(magical_vectors).T/py.sqrt(2),dims=[[2,2],[2,2]])
magical_basis_dag = magical_basis.dag()

def convert_to_magicalbasis(operator):
    if operator.type == 'oper':
        return magical_basis_dag * operator * magical_basis
    if operator.type == 'ket':
        return magical_basis_dag * operator
    if operator.type == 'bra':
        return operator * magical_basis

def convert_from_magicalbasis(operator):
    if operator.type == 'oper':
        return magical_basis * operator * magical_basis_dag
    if operator.type == 'ket':
        return magical_basis * operator
    if operator.type == 'bra':
        return operator * magical_basis
    
def calculate_UtU(operator):
    operator_magical = convert_to_magicalbasis(operator)
    transpose = operator_magical.full().T
    transpose = qp.Qobj(transpose,dims=operator.dims)
    utu = (transpose * operator_magical)
    return convert_from_magicalbasis(utu)
    
    

I must now solve the problem to find $U_A$ and $U_B$ such that, for a given maximally-entangled basis $\left\{ \left| \Psi_k \right\rangle\right\}$, one has
$$
\left( V_A \otimes V_B \right) e^{i\xi_k} \left| \Psi_k \right\rangle = \left| \Phi_k \right\rangle
$$
where $\left\{ \left| \Phi_k \right\rangle\right\}$ is the magical basis.

In [230]:
def make_vector_real(vector):
    phases = [cmath.phase(x)%py.pi for x in vector]
    diff = py.diff(phases + phases[0:1])
    assert min(abs(diff)) < 1e-8 , u"Vector can not be made real"
    phase = phases[0]
    vector = vector * py.exp(-1j*phase)
    return vector,phase

def finding_local_unitaries(entangled_basis):
    # Convert the basis vectors so that they are real in the magical basis
    real_vectors, phases = [],[]
    for vector in entangled_basis:
        real_vector,phase = make_vector_real(convert_to_magicalbasis(vector))
        real_vectors.append(real_vector)
        phases.append(phase)
        
    ket_ef = convert_from_magicalbasis(real_vectors[0] + 1j * real_vectors[1]) / py.sqrt(2)
    ket_epfp = convert_from_magicalbasis(real_vectors[0] - 1j * real_vectors[1]) / py.sqrt(2)

    ket_e = qp.ptrace(ket_ef,[0]).eigenstates()[1][-1]
    ket_f = qp.ptrace(ket_ef,[1]).eigenstates()[1][-1]
    ket_ep = qp.ptrace(ket_epfp,[0]).eigenstates()[1][-1]
    ket_fp = qp.ptrace(ket_epfp,[1]).eigenstates()[1][-1]
    # Must correct phases
    identity = qp.tensor([qp.qeye(2)]*2)
    phase_correction = identity.matrix_element(qp.tensor(ket_e,ket_f).dag(),ket_ef)
    ket_f = phase_correction * ket_f
    phase_correction = identity.matrix_element(qp.tensor(ket_ep,ket_fp).dag(),ket_epfp)
    ket_fp = phase_correction * ket_fp
        
    # Now I must rewrite the Psi_bar states in this basis
    local_basis = [qp.tensor(x,y) for x in [ket_e,ket_ep] for y in [ket_f,ket_fp]]
    matrix_t = []
    for base in real_vectors:
        base_comp = convert_from_magicalbasis(base)
        matrix_t.append([identity.matrix_element(x.dag(),base_comp) for x in local_basis])
    matrix = py.array(matrix_t).T
        
    # Find phase 'delta'
    delta = (py.pi/2 + cmath.phase(matrix[1,2]))%(2*py.pi)

    unitary_A = (qp.basis(2,0) * ket_e.dag() + qp.basis(2,1) * ket_ep.dag() * py.exp(+1j*delta))
    unitary_B = (qp.basis(2,0) * ket_f.dag() + qp.basis(2,1) * ket_fp.dag() * py.exp(-1j*delta))
    
    # Finding the phases to get magical basis
    # For some reason, the order of the magical basis must be changed
    phases = []
    transform = qp.tensor([unitary_A,unitary_B])
    changed_order = [magical_vectors[x] for x in [0,1,3,2]]
    for vector,magical in zip(entangled_basis,changed_order):
        magical_ket = qp.Qobj(py.array(magical).T,dims=[[2,2],[1,1]]) / py.sqrt(2)
        vec = transform * vector
        inner_product = identity.matrix_element(magical_ket.dag(),vec)
        phase = - cmath.phase(inner_product)
        phases.append( phase%(2*py.pi) )
    
    return unitary_A,unitary_B,phases

In [231]:
def decompositionCirac(unitary):
    # Step (i)
    utu = calculate_UtU(unitary)
    egvals, entangled_A = utu.eigenstates()
    epsilons = [0.5*(cmath.phase(x)%(2*py.pi)) for x in egvals]
    # Step (ii)
    Va, Vb, xis = finding_local_unitaries(entangled_A)
    # Step (iii)
    entangled_B = [py.exp(-1j*epsilon)*unitary*ket for ket,epsilon in zip(entangled_A,epsilons)]
    # Step (iv)
    Uad, Ubd, phases = finding_local_unitaries(entangled_B)
    Ua = Uad.dag()
    Ub = Ubd.dag()
    lambd = py.array(phases) - py.array(xis) - py.array(epsilons)
    Udiag = qp.tensor([Ua,Ub]).dag() * unitary * qp.tensor([Va,Vb]).dag()
    return Ua,Ub,Va,Vb,Udiag

### Testing

In [129]:
u = qp.rand_unitary(4,dims=[[2,2],[2,2]])
utu = calculate_UtU(u)
entangled = utu.eigenstates()[1]
UA,UB,phases = finding_local_unitaries(entangled)
print([py.exp(1j*ph)*qp.tensor(UA,UB)*x for x,ph in zip(entangled,phases)])

[Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.-0.70710678j]
 [0.+0.        j]
 [0.+0.        j]
 [0.+0.70710678j]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.+0.        j]
 [0.-0.70710678j]
 [0.-0.70710678j]
 [0.+0.        j]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.        ]
 [ 0.70710678]
 [-0.70710678]
 [ 0.        ]]]


In [130]:
unitary = qp.rand_unitary(4,dims=[[2,2],[2,2]])
Ua,Ub,Va,Vb,Udiag = decompositionCirac(unitary)
t = qp.tensor([Ua,Ub]).dag() * unitary * qp.tensor([Va,Vb]).dag()
print(convert_to_magicalbasis(t))
potential = qp.tensor([Ua,Ub]) * t * qp.tensor([Va,Vb])
print( unitary.dag() * potential )

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.81459775+0.5800263 j  0.        +0.        j  0.        +0.        j
   0.        +0.        j]
 [ 0.        +0.        j  0.82463191+0.56566971j  0.        +0.        j
   0.        +0.        j]
 [ 0.        +0.        j  0.        +0.        j -0.96810576-0.25054191j
   0.        +0.        j]
 [ 0.        +0.        j  0.        +0.        j  0.        +0.        j
   0.95392812+0.30003524j]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Implementing $N\left(\alpha,\beta,\gamma\right)$

Tackling the problem of implementing $N\left(\alpha,\beta,\gamma\right)$.

NEED ILLUSTRATOR!

In [166]:
def decompositionTwoQubits(unitary):
    phase = cmath.phase(scipy.linalg.det(unitary.full()))
#     unitary = unitary * py.exp(-1j*phase/4.)
    hamiltonian = qp.Qobj(1j * scipy.linalg.logm(unitary.full()),dims=[[2,2],[2,2]])
#     hamiltonian -= py.trace(hamiltonian) * py.eye(4)/4
    alphas = []
    alphas.append(qp.expect(qp.tensor([qp.sigmaz()]*2),hamiltonian))
    alphas.append(qp.expect(qp.tensor([qp.sigmax()]*2),hamiltonian))
    alphas.append(qp.expect(qp.tensor([qp.sigmay()]*2),hamiltonian))

    return hamiltonian, phase, alphas
    

In [167]:
unitary = qp.rand_unitary(4,dims=[[2,2],[2,2]])
Ua,Ub,Va,Vb,Udiag = decompositionCirac(unitary)
a,b,c=decompositionTwoQubits(Udiag)

In [170]:
c

[(0.358523802583407-1.2385925618474403e-15j),
 (-4.717703556311505+4.0245584642661925e-16j),
 (-0.7838530587158037+3.3306690738754696e-16j)]

In [ ]:
r = qp.Qobj(a,dims=[[2,2],[2,2]])
print(qp.expect(r,qp.tensor(qp.sigmaz(),qp.sigmaz())).real)
print(qp.expect(r,qp.tensor(qp.sigmax(),qp.sigmax())).real)
print(qp.expect(r,qp.tensor(qp.sigmay(),qp.sigmay())).real)
print(qp.expect(r,qp.tensor(qp.sigmaz(),qp.sigmax())).real)
print(qp.expect(r,qp.tensor(qp.sigmaz(),qp.sigmay())).real)
print(qp.expect(r,qp.tensor(qp.sigmax(),qp.sigmay())).real)
print(qp.expect(r,qp.tensor(qp.sigmax(),qp.sigmaz())).real)
print(qp.expect(r,qp.tensor(qp.sigmay(),qp.sigmax())).real)
print(qp.expect(r,qp.tensor(qp.sigmay(),qp.sigmaz())).real)
print(qp.ptrace(r,[0]))
print(qp.ptrace(r,[1]))

In [ ]:
hh = scipy.linalg.logm(a.full())

In [ ]:
hh.imag

## Implementing a one-qubit gate

This looks to be the easiest part, using a ZYZ Euler decomposition.

In [62]:
def keep_within_limits(theta, limits=None):
    """
    Keep an angle between +pi and -pi
    """
    if limits==None:
        limits = [-py.pi,+py.pi]
    if limits[1] < limits[0]:
        limits = limits[0:2][::-1]
    interval = limits[1] - limits[0]
    while (theta <= limits[0]):
        theta += interval
    while (theta > limits[1]):
        theta -= interval
    return theta

def convert_unitary_to_bloch(unitary):
    u0 = ((unitary).tr() / 2.0)
    u1 = ((unitary*qp.sigmax()).tr() * 1j / 2.0)
    u2 = ((unitary*qp.sigmay()).tr() * 1j / 2.0)
    u3 = ((unitary*qp.sigmaz()).tr() * 1j / 2.0)
    return (u0,u1,u2,u3)
    
    
    
def decompositionOneQubit(unitary, index = 0, qutip_test = False, optimize = False):
    phase = cmath.phase(scipy.linalg.det(unitary.full()))
    unitary = unitary * py.exp(-1j*phase/2.)
    u0,u1,u2,u3 = convert_unitary_to_bloch(unitary)
    
    t0Pt2by2 = math.atan2(u3.real,u0.real)
    t0Mt2by2 = math.atan2(u1.real,u2.real)
    t0 = (t0Pt2by2 + t0Mt2by2)
    t2 = (t0Pt2by2 - t0Mt2by2)
    cost1 = py.sqrt(u0.real**2 + u3.real**2) / 2.
    sint1 = py.sqrt(u1.real**2 + u2.real**2) / 2.
    t1 = 2 * math.atan2(sint1, cost1)
    
    if not qutip_test:
        optimize = False
        sq2 = py.sqrt(0.5)

        # No-Z gates
        if optimize and (equal(u0,sq2) and equal(u1,+sq2)):
            program = Progam()
            program.inst(gt.RX(+py.pi/2, index))
            yield program

            
        if optimize and (equal(u0,sq2) and equal(u1,-sq2)):
            program = Progam()
            program.inst(gt.RX(-py.pi/2, index))
            yield program
                
        # One-Z gates
        if optimize and (equal(u1,0) and equal(u2,0)):
            # Rotation around Z
            program = Progam()
            theta = keep_within_limits(2*math.atan2(u3,u0))
            program.inst(gt.RZ(theta, index))
            yield program
                
        if optimize and (equal(u1,0) and equal(u3,0)):
            theta = keep_within_limits(2*math.atan2(u2,u0))
            for x_correction in [False,True]:
                angles = [+py.pi/2, theta, -py.pi/2]
                if x_correction: 
                    angles[0] *= -1
                    angles[2] *= -1
                    angles[1] *= -1
                program = Progam()
                program.inst(gt.RX(angles[0], index))
                program.inst(gt.RZ(angles[1], index))
                program.inst(gt.RX(angles[2], index))
                yield program
                    
                
        if optimize and (equal(abs(u0),abs(u1)) and equal(abs(u2),abs(u3))):
            ### DO!
            yield None
            
        else:
            for z_correction,x_correction in itertools.product([False,True],repeat=2):
                angles = [t0, +py.pi/2, t1, -py.pi/2, t2]
                if z_correction:
                    angles[0] += - py.pi
                    angles[4] += + py.pi
                    angles[2] *= -1
                if x_correction:
                    angles[1] *= -1
                    angles[3] *= -1
                    angles[2] *= -1
                angles = [keep_within_limits(x) for x in angles]
                program = Program()
                program.inst(gt.RZ(angles[0], index))
                program.inst(gt.RX(angles[1], index))
                program.inst(gt.RZ(angles[2], index))
                program.inst(gt.RX(angles[3], index))
                program.inst(gt.RZ(angles[4], index))
                if optimize:
                    yield program
                else:
                    yield program,angles
                    
    else:
        # Testing with qutip
        rotationZ = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmaz()
        rotationX = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmax()
        unitaries = []
        unitaries.append(rotationZ(t0))
        unitaries.append(rotationX(+py.pi/2))
        unitaries.append(rotationZ(t1))
        unitaries.append(rotationX(-py.pi/2))
        unitaries.append(rotationZ(t2))
        implemented = unitaries[0]
        for unit in unitaries[1:]:
            implemented = unit * implemented
        yield implemented

The general case is done and it works.

TODO: optimize in cases where less Z rotations are necessary.

### Testing

In [63]:
def test_qutip():
    unitary = qp.rand_unitary(2)
    phase = cmath.phase(scipy.linalg.det(unitary.full()))
    unitary = unitary * py.exp(-1j*phase/2.)
#     prog = [x for x in decompositionOneQubit(unitary,qutip_test = True)][0]
    prog = decompositionOneQubit(unitary,qutip_test = True)
    test = unitary.dag() * next(prog)
    print(test)
test_qutip()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 1.]]


In [64]:
connection = api.QVMConnection()
def test_rigetti():
    unitary = qp.rand_unitary(2)
    phase = cmath.phase(scipy.linalg.det(unitary.full()))
    unitary = unitary * py.exp(-1j*phase/2.)
    progs = [x for x in decompositionOneQubit(unitary,qutip_test = False)]
    print(unitary)
    for prog,angle in progs:
        print('\nNew definition')
        print(prog)
        wavefunction = connection.wavefunction(prog,[])
        qutip_ket = unitary * qp.basis(2,0)
        print('Expected: ',qutip_ket)
        print('Result: ',wavefunction.pretty_print())
        rigetti_ket = qp.Qobj(wavefunction.amplitudes)
        print('Overlap: ', qp.qeye(2).matrix_element(rigetti_ket.dag(),qutip_ket))
        print('\n')
        wavefunction = connection.wavefunction(Program().inst(gt.X(0))+prog,[])
        qutip_ket = unitary * qp.basis(2,1)
        print('Expected: ',qutip_ket)
        print('Result: ',wavefunction.pretty_print())
        rigetti_ket = qp.Qobj(wavefunction.amplitudes)
        print('Overlap: ', qp.qeye(2).matrix_element(rigetti_ket.dag(),qutip_ket))
        print('\n')
test_rigetti()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.95956551+0.        j -0.26528929-0.09410428j]
 [ 0.26528929-0.09410428j  0.95956551+0.        j]]

New definition
RZ(0.34087641988010287) 0
RX(pi/2) 0
RZ(0.5706834784460747) 0
RX(-pi/2) 0
RZ(-0.34087641988010287) 0

Expected:  Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.95956551+0.        j]
 [0.26528929-0.09410428j]]
Result:  (0.96+0j)|0> + (0.27-0.09j)|1>
Overlap:  (1+1.3877787807814457e-17j)


Expected:  Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-0.26528929-0.09410428j]
 [ 0.95956551+0.        j]]
Result:  (-0.27-0.09j)|0> + (0.96+0j)|1>
Overlap:  (1.0000000000000002-1.0408340855860843e-17j)



New definition
RZ(0.34087641988010287) 0
RX(-pi/2) 0
RZ(-0.5706834784460747) 0
RX(pi/2) 0
RZ(-0.34087641988010287) 0

Expected:  Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.95956551+0.        j]
 [

In [65]:
keep_within_limits(py.pi/2 * 5 + 2)

-2.7123889803846897

In [66]:
rotationZ = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmaz()
rotationY = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmay()
convert_unitary_to_bloch(rotationY(-0.1))

((0.9987502603949663+0j), 0j, (-0.04997916927067833+0j), 0j)

In [181]:
## Implementing N(alpha,beta,gamma)
def implementingN(alpha,beta,gamma):
    rotationZ = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmaz()
    rotationY = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmay()
    rotationX = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmax()
    identity = qp.qeye(2)
    controlZ = qp.Qobj(py.diag([1,1,1,-1]),dims=[[2,2],[2,2]])
    for s0,s1,s2,a in itertools.product([+1,-1],repeat=4):
        unitaries = []
        angle1 = (py.pi/2 - 2*alpha) * s1*s0
        if a<0:
            angle1 += py.pi
        angle2 = 2 * gamma * s0
        angle3 = 2 * beta * s1
        unitaries.append(qp.tensor(rotationX(s0*py.pi/2),identity))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s1*py.pi/2),identity))
        unitaries.append(qp.tensor(rotationZ(angle1),identity))
        unitaries.append(qp.tensor(identity,rotationY(angle2)))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s2*py.pi/2),identity))
        unitaries.append(qp.tensor(identity,rotationY(angle3)))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s0*s1*s2*py.pi/2),identity))
        unitaries.append(qp.tensor(rotationZ(a*s0*s2*py.pi/2),identity))
        unitaries.append(qp.tensor(identity,rotationZ(a*py.pi/2)))
        unitary = unitaries[0]
        for next_unitary in unitaries[1:]:
            unitary = next_unitary * unitary
        yield unitary

In [188]:
import random
random.random()
alpha = random.random() * 2*py.pi
beta = random.random() * 2*py.pi
gamma = random.random() * 2*py.pi
hamiltonian = alpha*qp.tensor([qp.sigmaz()]*2)+beta*qp.tensor([qp.sigmax()]*2)+gamma*qp.tensor([qp.sigmay()]*2)
unitary = (-1j*hamiltonian).expm()
implementations = [x for x in implementingN(alpha,beta,gamma)]
test = [unitary.dag()*x for x in implementations]


In [306]:
def inverseProblem(theta1,theta2,theta3):
    rotationZ = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmaz()
    rotationY = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmay()
    rotationX = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmax()
    identity = qp.qeye(2)
    controlZ = qp.Qobj(py.diag([1,1,1,-1]),dims=[[2,2],[2,2]])

    for s0,s1,s2,s3,s4,s5 in itertools.product([+1,-1],repeat=6):
        unitaries = []
        unitaries.append(qp.tensor(rotationX(s0*py.pi/2),identity))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s1*py.pi/2),identity))
        unitaries.append(qp.tensor(rotationZ(theta1),identity))
        unitaries.append(qp.tensor(identity,rotationY(theta2)))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s2*py.pi/2),identity))
        unitaries.append(qp.tensor(identity,rotationY(theta3)))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s3*py.pi/2),identity))
        unitaries.append(qp.tensor(rotationZ(s5*py.pi/2),identity))
        unitaries.append(qp.tensor(identity,rotationZ(s4*py.pi/2)))
        unitary = unitaries[0]
        for next_unitary in unitaries[1:]:
            unitary = next_unitary * unitary
        yield unitary


In [301]:
units = [x for x in inverseProblem()]
get_hamiltonian = lambda x: scipy.linalg.logm(x.full()) * 1j
magical = [convert_to_magicalbasis(x) for x in units]
convert_to_magicalbasis(units[1])

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]]

In [302]:
is_diagonal = lambda x: py.norm(x - py.diag(py.diag(x))) < 1e-10
valid_indices = [i for i in range(len(magical)) if is_diagonal(magical[i].full())]

In [303]:
len(valid_indices)

16

In [304]:
for index in valid_indices:
#     print(bin(index)[2:].zfill(6),py.diag(magical[index].full()))
     print(bin(index)[2:].zfill(6),magical[index][0,0])

000001 (-0.9999999999999998+0j)
000010 (-0.9999999999999998+0j)
001100 -0.9999999999999998j
001111 0.9999999999999998j
010101 (0.9999999999999998+0j)
010110 (0.9999999999999998+0j)
011000 -0.9999999999999998j
011011 0.9999999999999998j
100100 -0.9999999999999998j
100111 0.9999999999999998j
101001 (0.9999999999999998+0j)
101010 (0.9999999999999998+0j)
110000 -0.9999999999999998j
110011 0.9999999999999998j
111101 (-0.9999999999999998+0j)
111110 (-0.9999999999999998+0j)


In [335]:
units_zero = [x for x in inverseProblem(0,0,0)]
magical_zero = [convert_to_magicalbasis(x) for x in units_zero]
phases = {}
for i in range(4):
    phases[i] = {index:magical_zero[index][i,i] for index in valid_indices}

In [362]:
units = [x for x in inverseProblem(0,0,0.1)]
magical = [convert_to_magicalbasis(x) for x in units]
for index in valid_indices:
    i = 0
    ph = py.conj(phases[i][index]) * magical[index][i,i]
    print(ph)

(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658-0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)
(0.9987502603949658+0.04997916927067831j)


In [363]:
phases[0]

{1: (-0.9999999999999998+0j),
 2: (-0.9999999999999998+0j),
 12: -0.9999999999999998j,
 15: 0.9999999999999998j,
 21: (0.9999999999999998+0j),
 22: (0.9999999999999998+0j),
 24: -0.9999999999999998j,
 27: 0.9999999999999998j,
 36: -0.9999999999999998j,
 39: 0.9999999999999998j,
 41: (0.9999999999999998+0j),
 42: (0.9999999999999998+0j),
 48: -0.9999999999999998j,
 51: 0.9999999999999998j,
 61: (-0.9999999999999998+0j),
 62: (-0.9999999999999998+0j)}

In [373]:
for i in valid_indices:
    print((phases[0][i]),(phases[2][i]))

(-0.9999999999999998+0j) 0.9999999999999998j
(-0.9999999999999998+0j) -0.9999999999999998j
-0.9999999999999998j (0.9999999999999998+0j)
0.9999999999999998j (0.9999999999999998+0j)
(0.9999999999999998+0j) -0.9999999999999998j
(0.9999999999999998+0j) 0.9999999999999998j
-0.9999999999999998j (0.9999999999999998+0j)
0.9999999999999998j (0.9999999999999998+0j)
-0.9999999999999998j (0.9999999999999998+0j)
0.9999999999999998j (0.9999999999999998+0j)
(0.9999999999999998+0j) -0.9999999999999998j
(0.9999999999999998+0j) 0.9999999999999998j
-0.9999999999999998j (0.9999999999999998+0j)
0.9999999999999998j (0.9999999999999998+0j)
(-0.9999999999999998+0j) 0.9999999999999998j
(-0.9999999999999998+0j) -0.9999999999999998j


In [408]:
## Implementing N(alpha,beta,gamma)
def implementingN_fromInverse(alpha,beta,gamma):
    rotationZ = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmaz()
    rotationY = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmay()
    rotationX = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmax()
    identity = qp.qeye(2)
    controlZ = qp.Qobj(py.diag([1,1,1,-1]),dims=[[2,2],[2,2]])
    for s0,s1,s2,s4 in itertools.product([+1,-1],repeat=4):
        s3 = s0*s1*s2
        s5 = -s0*s2*s4
        theta1 = -s0*s1*2*alpha - s1*s4*py.pi/2
        theta2 = 2 * gamma * s0
        theta3 = 2 * beta * s1
        unitaries = []
        unitaries.append(qp.tensor(rotationX(s0*py.pi/2),identity))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s1*py.pi/2),identity))
        unitaries.append(qp.tensor(rotationZ(theta1),identity))
        unitaries.append(qp.tensor(identity,rotationY(theta2)))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s2*py.pi/2),identity))
        unitaries.append(qp.tensor(identity,rotationY(theta3)))
        unitaries.append(controlZ)
        unitaries.append(qp.tensor(rotationX(s3*py.pi/2),identity))
        unitaries.append(qp.tensor(rotationZ(s2*s5*py.pi/2),identity))
        unitaries.append(qp.tensor(identity,rotationZ(s2*s4*py.pi/2)))
        unitary = unitaries[0]
        for next_unitary in unitaries[1:]:
            unitary = next_unitary * unitary
        yield unitary

In [449]:
## Implementing N(alpha,beta,gamma)
def implementingN_fromInverse(alpha,beta,gamma):
    rotationZ = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmaz()
    rotationY = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmay()
    rotationX = lambda x: py.cos(x/2) * qp.qeye(2) - 1j * py.sin(x/2) * qp.sigmax()
    identity = qp.qeye(2)
    controlZ = qp.Qobj(py.diag([1,1,1,-1]),dims=[[2,2],[2,2]])
    for s0,s1,s2,s4 in itertools.product([+1,-1],repeat=4):
        s3 = -s0*s1*s2
        s5 = s0*s2*s4
        theta1 = -s0*s1*2*alpha - s1*s4*py.pi/2
        theta2 = 2 * gamma * s0
        theta3 = 2 * beta * s1
        for r0,r1 in itertools.product([+1,-1],repeat=2):
            unitaries = []
            unitaries.append(qp.tensor(rotationX(s0*py.pi/2),identity))
            unitaries.append(controlZ)
            unitaries.append(qp.tensor(rotationX(s1*py.pi/2),identity))
            unitaries.append(qp.tensor(rotationZ(theta1),identity))
            unitaries.append(qp.tensor(identity,rotationX(+r0*py.pi/2)))
            unitaries.append(qp.tensor(identity,rotationZ(r0*theta2)))
            unitaries.append(qp.tensor(identity,rotationX(-r0*py.pi/2)))
            unitaries.append(controlZ)
            unitaries.append(qp.tensor(rotationX(-s2*py.pi/2),identity))
            unitaries.append(qp.tensor(identity,rotationX(+r1*py.pi/2)))
            unitaries.append(qp.tensor(identity,rotationZ(r1*theta3)))
            unitaries.append(qp.tensor(identity,rotationX(-r1*py.pi/2)))
            unitaries.append(controlZ)
            unitaries.append(qp.tensor(rotationX(s3*py.pi/2),identity))
            unitaries.append(qp.tensor(rotationZ(-s2*s5*py.pi/2),identity))
            unitaries.append(qp.tensor(identity,rotationZ(-s2*s4*py.pi/2)))
            unitary = unitaries[0]
            for next_unitary in unitaries[1:]:
                unitary = next_unitary * unitary
            yield unitary

In [450]:
import random
random.random()
alpha = random.random() * 2*py.pi
beta = random.random() * 2*py.pi
gamma = random.random() * 2*py.pi
hamiltonian = alpha*qp.tensor([qp.sigmaz()]*2)+beta*qp.tensor([qp.sigmax()]*2)+gamma*qp.tensor([qp.sigmay()]*2)
unitary = (-1j*hamiltonian).expm()
implementations = [x for x in implementingN_fromInverse(alpha,beta,gamma)]
test = [unitary.dag()*x for x in implementations]
test = [x/x[0,0] for x in test]

In [454]:
for x in test: print(x.full().real)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0